In [1]:
!pip install flask pyngrok
!pip install ultralytics
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 36.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import tensorflow
from flask import Flask
from pyngrok import ngrok
import PIL.Image
from io import BytesIO
from flask import Flask, request, jsonify
from pyngrok import ngrok
from ultralytics import YOLO
import requests
import easyocr

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
yolo_param_path = '/content/drive/MyDrive/bus_yolo.pt'
yolo_model = YOLO(yolo_param_path)

def change_to_image(image_data):
  image = PIL.Image.open(BytesIO(image_data))
  return image

def resize_image(image):
  return image.resize((640, 640))


def crop_image_yolo(image):
  result = yolo_model.predict(image)
  if result.__len__() == 0:
    return None
  box = result[0].boxes
  if len(box.xyxy) < 1:
    return None
  x1, y1, x2, y2 = map(int, box.xyxy[0])
  cropped_image = image.crop((x1, y1, x2, y2))

  return cropped_image

#If no detection, return "-1"
def OCR(cropped_image):
  cropped_image = cv2.cvtColor(np.array(cropped_image), cv2.COLOR_RGB2BGR)
  reader = easyocr.Reader(lang_list=['en'])
  result = reader.readtext(cropped_image, detail = 0, allowlist ='0123456789')
  print("Detected: ", result)
  if result == []:
    return '-1'
  return result


ngrok.set_auth_token("2R3LmYy4k5PTgZ57jKVgilmxSQy_7xzo63iws8uFBKNGXa3Lq")

In [5]:
#for debugging (display)
from IPython.display import Image

In [6]:
app = Flask(__name__)
ngrok.set_auth_token("2R3LmYy4k5PTgZ57jKVgilmxSQy_7xzo63iws8uFBKNGXa3Lq")
public_url = ngrok.connect(4000).public_url
@app.route("/")
def home():
  return f"Running Flask On Google Colab!"

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        print("ERROR!")
        return jsonify({'error': 'No file uploaded'})

    file = request.files['file']
    image_data = file.read()

    # Preprocessing Image
    #print("image come")    #FOR DEBUGGING
    image = change_to_image(image_data)
    #display(image)         #FOR DEBUGGING
    resized_image = resize_image(image)
    #display(resized_image) #FOR DEBUGGING


    ### YOLO start ###
    cropped_image = crop_image_yolo(resized_image)
    if (cropped_image == None):
      return "-1"
    #display(cropped_image) #FOR DEBUGGING
    ### YOLO end   ###

    ### OCR start ###
    bus_number = OCR(cropped_image)
    ### OCR end   ###

    # Prepare the response data
    return bus_number

print(f"To access click: {public_url}")
#Send public url to server
dictToSend = {'url' : public_url}
requests.post('https://blindroute-springboot.koyeb.app/url', data=dictToSend)

app.run(port=4000)

To access click: https://7fbc-34-135-22-151.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:4000
INFO:werkzeug:Press CTRL+C to quit



0: 640x640 (no detections), 1195.9ms
Speed: 22.6ms preprocess, 1195.9ms inference, 12.7ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [14/Dec/2023 13:27:37] "POST /upload HTTP/1.1" 200 -



0: 640x640 (no detections), 1016.1ms
Speed: 1.3ms preprocess, 1016.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


...

INFO:werkzeug:127.0.0.1 - - [14/Dec/2023 13:36:02] "POST /upload HTTP/1.1" 200 -



0: 640x640 (no detections), 1043.2ms
Speed: 1.7ms preprocess, 1043.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [14/Dec/2023 13:36:05] "POST /upload HTTP/1.1" 200 -



0: 640x640 (no detections), 1052.2ms
Speed: 2.5ms preprocess, 1052.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [14/Dec/2023 13:36:09] "POST /upload HTTP/1.1" 200 -
